# Build a Crossfiltering Dashboard

Learn how to create interactive dashboards with crossfiltering capabilities using HoloViews and Panel.

What is crossfiltering?
- Dynamic method of data visualization
- Interactive filtering across multiple dimensions
- Real-time updates across all visualizations
- Deeper insights into complex datasets

<video controls loop style="max-height: 400px; max-width: 100%;">
    <source src="https://assets.holoviz.org/panel/tutorials/panel_crossfilter_dashboard.mp4" type="video/mp4">
</video>

## Setup

Import required libraries and configure the environment:

In [ ]:
import holoviews as hv
import numpy as np
import pandas as pd
import panel as pn
from bokeh.models.formatters import NumeralTickFormatter

pn.extension(sizing_mode="stretch_width")

ACCENT = "teal"

SHORT_NAMES = {
    "Changzhou Railcar Propulsion Engineering Research and Development Center": "Changzhou",
    "Siemens Gamesa Renewable Energy": "Siemens Gamesa",
}

## Data Preparation

Create a function to load and preprocess the wind turbine data:

In [ ]:
@pn.cache()
def get_data():
    data = pd.read_csv("https://assets.holoviz.org/panel/tutorials/turbines.csv.gz")

    # Shorten manufacturer names for clarity
    mask = data.t_manu.isin(list(SHORT_NAMES))
    data.loc[mask, "t_manu"] = data.loc[mask, "t_manu"].map(SHORT_NAMES)
    
    # Convert capacity to gigawatts
    data.t_cap = data.t_cap / 10**6
    return data

# Load the data
data = get_data()

## Create Visualizations

Define a function to create the crossfiltering plots:

In [ ]:
# The @pn.cache decorator caches the output of this function to improve performance.
# It allows the function to return the same result faster if it is called again with the same inputs.
@pn.cache
def get_plots():
    # Get data from some external source or pre-processed function (not provided in the snippet).
    data = get_data()
    
    # Define a Holoviews Dataset for crossfiltering the data. This dataset allows easy manipulation,
    # aggregation, and visualization of data with interactive features.
    # It uses "t_manu", "p_year", and "t_cap" columns to define the data dimension and the variable.
    ds = hv.Dataset(data, ["t_manu", "p_year", "t_cap"], "t_cap")
    
    # Aggregate data by year ("p_year"). The `function=np.sum` specifies that the aggregation 
    # should sum up all values for each year. Then it sorts the data by "p_year".
    # The slice [1995:] only takes data from 1995 onwards.
    ds_by_year = ds.aggregate("p_year", function=np.sum).sort("p_year")[1995:]
    
    # Aggregate data by manufacturer ("t_manu"), summing the values. Then sort by capacity ("t_cap")
    # in descending order. Finally, only keep the top 20 manufacturers (using .iloc[:20]).
    ds_by_manufacturer = ds.aggregate("t_manu", function=np.sum).sort(
        "t_cap", reverse=True
    ).iloc[:20]
    
    # Configure a formatter for the y-axis ticks to use a comma for thousands separators.
    formatter = NumeralTickFormatter(format="0,0")
    
    # Create a bar plot by year.
    plot_by_year = hv.Bars(
        ds_by_year, ("p_year", "Year"), ("t_cap", "Capacity (GW)")  # Define x and y axes labels
    ).opts(
        responsive=True,  # Make the plot responsive to the container size.
        min_height=300,  # Set a minimum height for the plot.
        yformatter=formatter,  # Format the y-axis ticks to include commas.
        color=ACCENT,  # Set the color of the bars (variable named ACCENT).
        tools=["hover"],  # Add hover tool to show additional data when the user hovers over bars.
        active_tools=["box_select"],  # Enable box select for selecting a subset of bars interactively.
    )
    
    # Create a bar plot by manufacturer.
    plot_by_manufacturer = hv.Bars(
        ds_by_manufacturer, ("t_manu", "Manufacturer"), ("t_cap", "Capacity (GW)")  # Define x and y axes labels
    ).opts(
        responsive=True,  # Make the plot responsive to the container size.
        min_height=300,  # Set a minimum height for the plot.
        xrotation=90,  # Rotate x-axis labels by 90 degrees for better readability.
        yformatter=formatter,  # Format the y-axis ticks to include commas.
        color=ACCENT,  # Set the color of the bars (variable named ACCENT).
        tools=["hover"],  # Add hover tool to show additional data when the user hovers over bars.
        active_tools=["box_select"],  # Enable box select for selecting a subset of bars interactively.
    )

    # Return the plots stacked vertically in a single-column layout.
    return (plot_by_year + plot_by_manufacturer).cols(1)


## Enable Crossfiltering

Link the selections between plots to enable crossfiltering:

In [ ]:
# Create linked plots
crossfilter_plots = hv.link_selections(get_plots()).opts(shared_axes=False)

# Create dashboard
dashboard = pn.template.FastListTemplate(
    title="Windturbine Dashboard with Crossfiltering",
    main=[crossfilter_plots],
    main_layout=None,
    accent=ACCENT,
)

dashboard.servable();

## Key Features

1. **Interactive Selection**
   - Box select tool enabled
   - Hover tooltips for details
   - Real-time updates

2. **Data Views**
   - Capacity by Year
   - Capacity by Manufacturer
   - All views linked through crossfiltering

3. **Responsive Design**
   - Adapts to window size
   - Clean, modern interface
   - Consistent styling

### Complete Code

```{include} ./scripts/crossfiltering.py
:code: python
```

To run the code, type the following command in your terminal:

```bash
panel serve crossfiltering.py --dev
```

## Important Notes

1. **Shared Dataset**
   - Required for linked brushing
   - All plots must use same HoloViews Dataset

2. **Session Management**
   - Create new `crossfilter_plots` for each session
   - Sessions cannot share `link_selections` objects

3. **Performance**
   - Use `@pn.cache` for data loading
   - Keep data transformations efficient
